# Units

In Synthesizer all quantities a user interacts with (that are not dimensionless) have units associated with them. We implement this unit system using the ``unyt`` package. 

Synthesizer objects and methods should always be provided with quantites and associated units. This can be easily achieved with the unyt package.

In [ ]:
from unyt import Mpc

# Define a variable with units
x = 1 * Mpc

print(x)
print("x is now a unyt_quantity: type(x)=", type(x))

All unit functionality in synthesizer is contained in the `units` module. This module contains an importable object containing the default units of all attributes throughout synthesizer.

In [ ]:
from synthesizer.units import default_units

print(default_units)

## The `Units` object

The unit system is defined by the `Units` object. This object contains a collection of attributes defining the units associated to each "category" of quantity throughout Synthesizer. 

Importantly, `Units` is a `Singleton` object. This means there can only ever be one instance of `Units`; if a second is instantiated then the first is returned. This ensures that the unit system remains consistent when running Synthesizer.

In [ ]:
from synthesizer.units import Units

# Define multiple Units instances
units1 = Units()
units2 = Units()

print("Both units instances are the same object:", units1 is units2)

You can take a look at the unit system by printing the instance of `Units`.

In [ ]:
print(units1)

## Modifying the default unit system

If the default unit system works for your needs then you don't need to do anything. You will never interact with the `Units` object and all quantites will have the default units associated to them automatically. However, if you need to change one or more of the units used you can import `Units` and instantiate it with a dictionary of the modified quantities.

This dictionary of modified quantities can either modify an existing category or it can defining a unit for a specific attribute. We demonstrate this below by modifying the default unit system to use `kpc` for the ``"spatial"`` category, but also override this to use `Mpc` for coordinates and `Myr` for ages specifically.


In [ ]:
# Ensure warnings are printed
import warnings

from unyt import Mpc, Msun, Myr, kpc

warnings.simplefilter("always")

# Make the dictionary containing the units we want to change
new_units = {
    "spatial": kpc,
    "coordinates": Mpc,
    "ages": Myr,
}

# Set up the modified unit system
units = Units(new_units)

print()
print(units)

You'll notice that something has gone wrong here... but recall that the unit system will return the original if one exists, so actually this should be completely expected.

This issue highlights the need to set up `Units` **before** doing anything else. If any computations have been done the `Units` instance will exist and will not be modifiable after the fact. However, should you fall in this trap the code will warn you as above - no hidden gotchas here!

Now, lets go against the advice above, and use the **highly inadvisable** force argument to get a new Unit system. But please note, **in a real use case, forcing a modified unit system WILL NOT convert existing quantities to the new unit system**.

In [ ]:
# Set up the modified unit system
units = Units(new_units, force=True)

print()
print(units)

### Permenantly modify the default unit system

If you want to permenantly modify the default unit system you can do so by first modifying the `Units` object and then calling the ``overwrite_defaults_yaml`` method. This will write the modified unit system to the default units file. 

Note, you can also explicitly edit the ``default_units.yml`` file within the source code.

In [ ]:
units.overwrite_defaults_yaml()

When the above function is called we also write out the original default units system if it hasn't already been written out. This ensures the unit redefinition is reversible. To revert to the original unit system you can call the ``reset_defaults_yaml`` method.

In [ ]:
units.reset_defaults_yaml()
print(units)

## Working with `Quantity` objects

There is no need to work with the `Units` object itself beyond initially defining a modified unit system. Beyond this, all unit operations are handled "behind the scenes". This hidden functionality is enabled by the `Quantity` object. 

All attributes on Synthesizer objects which carry units are in fact `Quantity` objects. `Quantity` objects carry a the unit of the attribute (extracted from the global unit system), and extract the appropriate units depending on the name of the variable storing the `Quantity`. As such, a user will never instantiate a quantity themselves, but their usage is important. 

One simple thing to keep in mind is how to return the value with or without units. This is achieved by the application or omission of a leading underscore to a variable name. 

Lets create an `Sed` object, which has a wavelength array stored under `lam`. 

In [ ]:
import numpy as np
from unyt import Hz, angstrom, erg, s

from synthesizer.emissions import Sed

# Make an sed with arbitrary arguments
sed = Sed(
    lam=np.linspace(10, 1000, 10) * angstrom, lnu=np.ones(10) * erg / s / Hz
)

We can access this attribute with units as you would expect to access any attribute.

In [ ]:
print(sed.lam)

Or we can append a leading underscore and return it without units.

In [ ]:
print(sed._lam)

In the case of compound units this is somewhat less elegant. Lets demonstrate with a `Stars` object.

In [ ]:
from synthesizer.particle.stars import Stars

# Create a dummy Stars object
stars = Stars(
    initial_masses=np.random.rand(10) * Msun,
    ages=np.ones(10) * Myr,
    metallicities=np.ones(10),
)

If we print the `initial_masses` with units we get the compound version in `kg`.

In [ ]:
print(stars.initial_masses)

However, if we extract the values alone we get the values we expect in $M_\odot$.

In [ ]:
print(stars._initial_masses)

Its worth keeping this in mind whenever extracting masses from synthesizer objects.

### Automatic unit conversion

Finally, let's utilise some automatic unit conversion. If we input a mixture of properties to a synthesizer object, all with different units to the global unit system, we don't have to convert them all before inputting them. As long as we pass them to synthesizer with *unyt* units attached, the conversion will be handled automatically. Here we use a `Stars` object again to demonstrate.

In [ ]:
from unyt import Mpc, g, m

# Create a dummy Stars object
stars = Stars(
    initial_masses=np.random.rand(10) * 10**34.0 * g,
    ages=np.ones(10) * Myr,
    metallicities=np.ones(10),
    coordinates=np.random.rand(10, 3) * Mpc,
    smoothing_lengths=np.random.rand(10) * 10**22.0 * m,
)

print(
    "stars.initial_masses[0]=",
    stars.initial_masses[0],
    "=",
    stars._initial_masses[0],
    "Msun",
)
print("stars.ages[0]=", stars.ages[0])
print("stars.coordinates[0]=", stars.coordinates[0])
print("stars.smoothing_lengths[0]=", stars.smoothing_lengths[0])